In [1]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET

### Create a code block to repeat
This block of code is boilerplate code that needs to be repeated for each parameter,

In [2]:
code_block = """case('{0}')
    this%{0}%name = last
    call this%{0}%read(iUnit)
    line = line + this%{0}%size() + this%{0}%dim_names%size() + ENTRY_OFFSET"""

### Get list of sorted parameters 
These parameters were generated by Generate_PRMS_valid_parameters_by_module-Copy1 and copied here.

In [3]:
workfile = '/Users/pnorton/Projects/National_Hydrology_Model/src/fortran/prms6/src/xml/parameters.xml'

xml_tree = xmlET.parse(workfile)
xml_root = xml_tree.getroot()

print(xml_root.tag)

parameters


In [4]:
param_dict = {}

# Iterate over child nodes of root
for elem in xml_root.findall('parameter'):
    # print(elem.attrib.get('name'))
    name = elem.attrib.get('name')
    dtype = elem.find('type').text
    
    # Print the elements of the node
    # print(elem.find('desc').text)
    # print(elem.find('size').text)
    # dim_size = int(elem.find('size').text)
    param_dict[name] = dtype
    
parameters = param_dict.keys()
parameters.sort()


In [5]:
# Open files to write the typedefs and code blocks to
basedir = '/Users/pnorton/Projects/National_Hydrology_Model/src/fortran/prms6'
typedef_hdl = open('{}/src/prmslib/classes/sm_parameter_typedef.inc'.format(basedir), 'w')
code_hdl = open('{}/src/prmslib/classes/sm_parameter_case_block.inc'.format(basedir), 'w')

tb = '  '

# Write the start of the select-case statement
code_hdl.write('select case(buf)\n')

code_footer = """
  case default
    write(output_unit, *) 'WARNING: Skipping parameter, ', last

    ! Get the number of dimension names
    read(iUnit, *) N
    line = line + 1

    do ii=1, N
      read(iUnit, '(A)')  ! Skip the dimension names
      line = line + 1
    enddo

    read(iUnit, *) N  ! Get the number of entries for parameter
    line = line + 1
    read(iUnit, *)    ! Skip the datatype
    line = line + 1

    do ii=1, N
      read(iUnit, *)  ! Skip the parameter entries

      ! if (istat /= 0) then
      !   inquire(UNIT=iUnit, NAME=filename)
      !   write(output_unit, *) "ERROR: Reading from file: " // trim(filename)
      !   close(iUnit)
      !   stop
      ! endif

      line = line + 1
    enddo
end select"""

### Generate the repeated code

In [6]:
for pp in parameters:
    code_hdl.write('{}{}\n'.format(tb, code_block.format(pp)))
#     print(code_block.format(pp))
    
code_hdl.write(code_footer)
code_hdl.write('\n')
code_hdl.close()

### Create code blocks for defining the parameter datatypes
This uses a parameter to datatype mapping which was generated by Generate_PRMS_valid_parameters_by_module-Copy1.

In [7]:
datatype_map = {'I': 'iVariable', 'F': 'rVariable', 'S':'sVariable'}

### Define the code block for parameter type definitions

In [8]:
# type(rVariable) :: K_coef
def_block = """type({}) :: {}"""

### Generate the type definition code blocks

In [9]:
for pp in parameters:
    typedef_hdl.write('{}\n'.format(def_block.format(datatype_map[param_dict[pp]], pp)))
#     print(def_block.format(datatype_map[param_dict[pp]], pp))
typedef_hdl.write('\n')
typedef_hdl.close()